#  Système de Recommandation de Films

Ce notebook démontre un système de recommandation de films utilisant le filtrage collaboratif basé sur la similarité cosinus.  
Nous allons :
- Charger et prétraiter les données des films et des notes des utilisateurs.
- Calculer la similarité entre les films en fonction des genres.
- Calculer la similarité entre les utilisateurs en fonction de leurs évaluations.
- Implémenter un système de recommandation hybride combinant filtrage collaboratif et basé sur le contenu.
- Construire une interface simple avec Streamlit pour interagir avec le système.

In [13]:
import pandas as pd
import re
import requests
from sklearn.metrics.pairwise import cosine_similarity
import streamlit as st
from sklearn.preprocessing import MultiLabelBinarizer
import IPython.display as display
from PIL import Image
import urllib

##  Chargement des données
Nous chargeons deux fichiers CSV :
- `movies.csv` : contient les informations sur les films (titres, genres, etc.).
- `ratings.csv` : contient les notes attribuées par les utilisateurs aux films.

In [14]:
movies = pd.read_csv("/Users/ousmanediallo/Downloads/movies.csv")
ratings = pd.read_csv("/Users/ousmanediallo/Downloads/ratings.csv")

##  Prétraitement des genres des films
Nous transformons les genres en listes et les binarisons pour obtenir une matrice exploitable.

In [15]:
movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))
mlb = MultiLabelBinarizer()
genre_matrix = mlb.fit_transform(movies['genres'])
genre_df = pd.DataFrame(genre_matrix, columns=mlb.classes_, index=movies['movieId'])

##  Calcul de la similarité entre films
Nous utilisons la similarité cosinus pour comparer les films en fonction de leurs genres.

In [16]:
content_similarity = cosine_similarity(genre_df)
content_similarity_df = pd.DataFrame(content_similarity, index=movies['movieId'], columns=movies['movieId'])


##  Création d'une matrice utilisateur-film
Nous structurons les notes attribuées par les utilisateurs sous forme de matrice.


In [17]:
user_movie_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')
user_movie_matrix.fillna(0, inplace=True)


##  Calcul de la similarité entre utilisateurs
Nous utilisons la similarité cosinus pour comparer les utilisateurs en fonction de leurs notes.


In [18]:
user_similarity = cosine_similarity(user_movie_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

##  Fonction pour nettoyer les titres de films
Permet de supprimer ', The' et l'année de sortie d'un titre de film.

In [19]:
def clean_movie_title(title):
    title = re.sub(r'\s*\(\d{4}\)$', '', title)  # Supprimer l'année
    match = re.match(r'^(.*), (The|A|An)$', title)
    return f"{match.group(2)} {match.group(1)}" if match else title

##  Fonction pour récupérer l'affiche d'un film
Nous utilisons l'API OMDb pour récupérer les affiches des films.

In [20]:
def get_movie_poster(title):
    api_key = "1d34d942"
    cleaned_title = clean_movie_title(title)
    url = f"http://www.omdbapi.com/?t={cleaned_title}&apikey={api_key}"
    response = requests.get(url).json()
    return response["Poster"] if "Poster" in response and response["Poster"] != "N/A" else "https://via.placeholder.com/150"

##  Système de recommandation hybride
Combine le filtrage collaboratif et basé sur le contenu pour recommander des films.

In [21]:
def HybridRecommender(user_id, top_n=3):
    similar_users = user_similarity_df[user_id].drop(user_id)
    similar_users = similar_users[similar_users > 0]
    if similar_users.empty:
        return []

    movie_scores = {}
    for similar_user, similarity in similar_users.items():
        rated_movies = user_movie_matrix.loc[similar_user][user_movie_matrix.loc[similar_user] > 0]
        for movie, rating in rated_movies.items():
            movie_scores[movie] = movie_scores.get(movie, 0) + rating * similarity

    movie_scores = pd.Series(movie_scores).sort_values(ascending=False)
    user_watched_movies = user_movie_matrix.loc[user_id][user_movie_matrix.loc[user_id] > 0].index
    recommended_movie_ids = [movie for movie in movie_scores.index if movie not in user_watched_movies][:top_n]

    content_scores = {}
    for movie_id in recommended_movie_ids:
        similar_movies = content_similarity_df[movie_id].sort_values(ascending=False)
        similar_movies = similar_movies[similar_movies.index != movie_id].head(top_n)
        for similar_movie, similarity in similar_movies.items():
            if similar_movie not in recommended_movie_ids:
                content_scores[similar_movie] = content_scores.get(similar_movie, 0) + similarity

    hybrid_scores = pd.Series({**movie_scores.to_dict(), **content_scores}).sort_values(ascending=False)
    recommended_movies = movies[movies['movieId'].isin(hybrid_scores.index[:top_n])][['title', 'movieId']]

    return [(row['title'], get_movie_poster(row['title'])) for _, row in recommended_movies.iterrows()]

##  Interface utilisateur avec Streamlit
Interface simple permettant de générer des recommandations en entrant un ID utilisateur.

In [22]:
st.title("🎬 Système de Recommandation de Films")
user_id = st.number_input("Entrez votre ID utilisateur :", min_value=1, step=1)
if st.button("Recommander"):
    with st.spinner("Génération des recommandations..."):
        recommendations = HybridRecommender(user_id)
    if recommendations:
        st.subheader("📌 Films recommandés :")
        for title, poster_url in recommendations:
            st.image(poster_url, caption=title, width=200)
    else:
        st.warning("Aucune recommandation disponible pour cet utilisateur.")

2025-03-28 10:09:00.589 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-28 10:09:00.591 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-28 10:09:00.592 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-28 10:09:00.593 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-28 10:09:00.594 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-28 10:09:00.595 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-28 10:09:00.596 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-28 10:09:00.596 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

##  Affichage des affiches de films sans Streamlit
Utilisation de IPython pour afficher les affiches directement dans le notebook.

In [ ]:
if recommendations:
    for title, poster_url in recommendations:
        image = Image.open(urllib.request.urlopen(poster_url))
        display.display(image)
        print(title)